## Azure Machine Learning Engineer
### Project 1 - Optimising an ML Pipeline in Azure

#### Step 1 Create a workspace
In this step we are making an Azure Workspace and setting up an experiment

In [10]:
# Create a new workspace and define an experiment.

from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
ws.get_details()

# Choose a name for the experiment
experiment_name = 'udacity-project'
experiment = Experiment(workspace=ws, name = experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-141976
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-141976


### Setup Compute
Create a new compute or use an existing one if its present

In [11]:
# Create a compute cluster to provision VM Resources.

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = 'cpu-cluster-01'

#Verify that the culster does not exist already
try:
    compute_target = ComputeTarget(workspace = ws, name = cpu_cluster_name)
    print('Found existing cluster, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True)

Found existing cluster, use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [24]:
# Setup Hyperparameter Tuning with Hyperdrive.

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os
import shutil
from azureml.core import ScriptRunConfig

#Define the parameter search space/method
# Specify parameter sampler, in this case we are looking to get defined ranges and pass back to the SKILEARN
# training model.

ps = RandomParameterSampling({
    '--C': choice(1.0, 0.1, 0.05),
    '--max_iter': choice(50,100,150)})

#note in the script, C is described as float
#note in the script, max iter is described as integer

# Specify an early termination Policy
# Other options are median policy, have stuck with bandit for simplification
# Bandit policy stops if its less than 10% of best model, starting and interval 5
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#creates a training director.
if "training" not in os.listdir():
    os.mkdir("./training")
    
script_folder = './training'

os.makedirs(script_folder, exist_ok = True)

shutil.copy('./train.py',script_folder)

# Create a SKLearn estimator for use with train.py ### YOUR CODE HERE ###
est = SKLearn(source_directory = script_folder,
              entry_script ='train.py',
              compute_target = compute_target,
              vm_size = 'Standard_d2_v')

hyperdrive_config = HyperDriveConfig(estimator = est,
                                     hyperparameter_sampling = ps,
                                     policy = early_termination_policy,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 50,
                                     max_concurrent_runs = 4)

In [25]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment

#experiment = Experiment(ws, experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_config, show_output = True)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [26]:
import joblib
# Get your best run and save the model from that run.

best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
#best_hyperdrive_run_metrics = best_hyperdrive_run.get_metrics()

print("Best Run Metrics :", best_hyperdrive_run.get_metrics())

best_hyperdrive_run.download_file(
    best_hyperdrive_run.get_file_names()[-1],
    output_file_path="./outputs/"
)
best_hyperdrive_model = best_hyperdrive_run.register_model(
    model_name="best_hyperdrive_model",
    model_path="./outputs/best_hyperdrive_model.joblib",
    tags=best_hyperdrive_run.get_metrics()
)

Best Run Metrics : {'Regularization Strength:': 0.05, 'Max iterations:': 150, 'Accuracy': 0.9097622660596864}


### Now compare against AutoML

In [46]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

#  path to URL from Chrome DevTools Console
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

#  read remote URL data to DataFrame
ds = TabularDatasetFactory.from_delimited_files(url)

In [54]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#automl settings have the optios to assign the dataframe and identify a target variable within it.
datafinal = pd.concat([x,y], axis = 1)
datafinal.head()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [55]:
#need to create a tabular dataset for the AutoML Config
#Convert to CSV
#datafinal.to_csv('./train/dataset.csv')
#datastore = ws.get_defaultdatastore()
#datastore.upload(src_director = './train', target_path = 'train_data')

#data_tab = Dataset.Tabular.from_delimited_files(path = [(datastore, ('train_data/dataset.csv'))])

In [56]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    compute_target = compute_target,
    primary_metric = 'accuracy',
    n_cross_validations = 5,
    task = 'classification',
    X = x_train,
    y = y_train,
    enable_onnx_compatible_models = True)

In [58]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    compute_target = compute_target,
    primary_metric = 'accuracy',
    n_cross_validations = 5,
    task = 'classification',
    training_data = datafinal,
    label_column_name = 'y',
    enable_onnx_compatible_models = True)

In [57]:
# Submit your automl run
experiment = Experiment(ws, "udacity_automl")
automl_run = experiment.submit(config = automl_config, show_output = True)

ConfigException: ConfigException:
	Message: Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "X",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_error": {
                    "code": "InvalidInputDatatype"
                }
            }
        }
    }
}

## Clean Up Resources
To ensure we don't continue to acrue cost for the resources, delete resources.

#### Delete Compute Instance and Cluster
1. In the Microsoft Azure Machine Learning Portal, select Compute on the far left.

2. From the list, select the compute instances and or compute clsuters you created.

3. Select Delete.

In [ ]:
compute_target.delete()
compute_target.wait_for_completion(show_output=True)